# CRWU Data Analytics Bootcamp - Project 2: ETL

# Step 1: EXTRACT

### Dependencies

In [29]:
import pandas as pd
from sqlalchemy import create_engine

### Data Extraction

In [26]:
#Load data CSVs to their own dataframes using Pandas
data_df = pd.read_csv('data/daily.csv')
state_df = pd.read_csv('data/us_state_population.csv')

# Step 2: TRANSFORM

### Transform state population CSV by removing fields, adding state abbreviations, and renaming columns

In [52]:
#Drop the state column that had numbers
state_df = state_df.drop(columns=['state'])

#Drop Puerto Rico data from dataframe
state_df = state_df[state_df.name !="Puerto Rico"]

#Make a list of the abbreviations of the states and District of Columbia
state = ['AL','AK','AZ','AR','CA','CO','CT','DE','DC','FL','GA','HI','ID','IL','IN','IA','KS','KY','LA','ME','MD','MA','MI','MN','MS','MO','MT','NE','NV','NH','NJ','NM','NY','NC','ND','OH','OK','OR','PA','RI','SC','SD','TN','TX','UT','VT','VA','WA','WV','WI','WY']

#Add new state column to df using the alphabetical list of states
state_df['state'] = state

#Reorder the columns
state_df = state_df[['state','name','popestimate2019']]

#Rename column
state_df= state_df.rename(columns={"popestimate2019":"pop_2019"})

state_df

KeyError: "['popestimate2019'] not in index"

In [28]:
#Save the population data to a new CSV
state_df.to_csv('us_state_pop.csv', index=False)

In [6]:
data_df.head()

,date,state,positive,negative,pending,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,inIcuCumulative,onVentilatorCurrently,...,hospitalized,total,totalTestResults,posNeg,fips,deathIncrease,hospitalizedIncrease,negativeIncrease,positiveIncrease,totalTestResultsIncrease
0,20200421,AK,329.0,10790.0,NaN,42.0,36.0,NaN,NaN,NaN,...,36.0,11119.0,11119.0,11119.0,2,0.0,0.0,987.0,8.0,995.0
1,20200421,AL,5231.0,43295.0,NaN,NaN,699.0,NaN,260.0,NaN,...,699.0,48526.0,48526.0,48526.0,1,10.0,58.0,2420.0,206.0,2626.0
2,20200421,AR,2227.0,25214.0,NaN,86.0,291.0,NaN,NaN,27.0,...,291.0,27441.0,27441.0,27441.0,5,1.0,0.0,584.0,304.0,888.0
3,20200421,AS,0.0,3.0,17.0,NaN,NaN,NaN,NaN,NaN,...,NaN,20.0,3.0,3.0,60,0.0,0.0,0.0,0.0,0.0
4,20200421,AZ,5251.0,49901.0,NaN,637.0,NaN,285.0,NaN,181.0,...,NaN,55152.0,55152.0,55152.0,4,21.0,0.0,465.0,187.0,652.0


In [7]:
#Make a new dataframe using only the columns we want to see
covid_df = data_df[['date','state','positive','total','death']]

#Rename columns for clarity
covid_df = covid_df.rename(columns={'positive':'positive_tests','total':'total_tests'})

covid_df

,date,state,positive_tests,total_tests,death
0,20200421,AK,329.0,11119.0,9.0
1,20200421,AL,5231.0,48526.0,177.0
2,20200421,AR,2227.0,27441.0,43.0
3,20200421,AS,0.0,20.0,NaN
4,20200421,AZ,5251.0,55152.0,208.0
...,...,...,...,...,...
2615,20200301,MI,9.0,9.0,NaN
2616,20200301,RI,1.0,1.0,NaN
2617,20200301,WA,30.0,30.0,8.0
2618,20200229,WA,18.0,18.0,5.0


In [58]:
#Drop the columns for the countries outside of the 50 states and DC
covid_df = covid_df[covid_df.state !='AS']
covid_df = covid_df[covid_df.state !='GU']
covid_df = covid_df[covid_df.state !='MP']
covid_df = covid_df[covid_df.state !='PR']

covid_df

,date,state,positive_tests,total_tests,death
0,20200421,AK,329.0,11119.0,9.0
1,20200421,AL,5231.0,48526.0,177.0
2,20200421,AR,2227.0,27441.0,43.0
4,20200421,AZ,5251.0,55152.0,208.0
5,20200421,CA,33261.0,300100.0,1268.0
...,...,...,...,...,...
2615,20200301,MI,9.0,9.0,NaN
2616,20200301,RI,1.0,1.0,NaN
2617,20200301,WA,30.0,30.0,8.0
2618,20200229,WA,18.0,18.0,5.0


In [59]:
#Save to a new CSV
covid_df.to_csv('daily_covid_stats.csv', index=False)

In [8]:
#Merge tables on state abbreviation. Inner merge to add state names and population to every column
combined_df = pd.merge(covid_df, state_df, on='state')

combined_df.head()

,date,state,positive_tests,total_tests,death,name,2019_pop
0,20200421,AK,329.0,11119.0,9.0,Alaska,731545
1,20200420,AK,321.0,10124.0,9.0,Alaska,731545
2,20200419,AK,319.0,9895.0,9.0,Alaska,731545
3,20200418,AK,314.0,9655.0,9.0,Alaska,731545
4,20200417,AK,309.0,9450.0,9.0,Alaska,731545


In [9]:
#Sort the values by the date and state name, in descending order by date and ascending order by name
combined_df = combined_df.sort_values(['date', 'name'], ascending =(False,True))

In [10]:
#Add a column for percentage tested vs the total population for each state
combined_df['pct_tested'] = round((combined_df['total_tests']/combined_df['2019_pop']) *100,2)

#Add a column for total cases per million
combined_df['tested_per_million'] = round((combined_df['total_tests']/combined_df['2019_pop'])*1000000,0)
combined_df.head(10)

,date,state,positive_tests,total_tests,death,name,2019_pop,pct_tested,tested_per_million
47,20200421,AL,5231.0,48526.0,177.0,Alabama,4903185,0.99,9897.0
0,20200421,AK,329.0,11119.0,9.0,Alaska,731545,1.52,15199.0
140,20200421,AZ,5251.0,55152.0,208.0,Arizona,7278717,0.76,7577.0
93,20200421,AR,2227.0,27441.0,43.0,Arkansas,3017804,0.91,9093.0
189,20200421,CA,33261.0,300100.0,1268.0,California,39512223,0.76,7595.0
238,20200421,CO,10106.0,47466.0,449.0,Colorado,5758736,0.82,8242.0
286,20200421,CT,20360.0,64192.0,1423.0,Connecticut,3565287,1.80,18005.0
380,20200421,DE,2931.0,16656.0,82.0,Delaware,973764,1.71,17105.0
332,20200421,DC,3098.0,14939.0,112.0,District of Columbia,705749,2.12,21168.0
427,20200421,FL,27495.0,278262.0,856.0,Florida,21477737,1.30,12956.0


In [11]:
#Reorder columns for clarity
combined_df = combined_df[['date','state','name','positive_tests','death','total_tests','2019_pop','pct_tested','tested_per_million']]
combined_df.head()

,date,state,name,positive_tests,death,total_tests,2019_pop,pct_tested,tested_per_million
47,20200421,AL,Alabama,5231.0,177.0,48526.0,4903185,0.99,9897.0
0,20200421,AK,Alaska,329.0,9.0,11119.0,731545,1.52,15199.0
140,20200421,AZ,Arizona,5251.0,208.0,55152.0,7278717,0.76,7577.0
93,20200421,AR,Arkansas,2227.0,43.0,27441.0,3017804,0.91,9093.0
189,20200421,CA,California,33261.0,1268.0,300100.0,39512223,0.76,7595.0


In [12]:
#Save dataframe to CSV
combined_df.to_csv('final_covid_data.csv',index=False)

# Step 3: LOAD

In [42]:
state_df.head()

,state,name,2019_pop
0,AL,Alabama,4903185
1,AK,Alaska,731545
2,AZ,Arizona,7278717
3,AR,Arkansas,3017804
4,CA,California,39512223


In [45]:
conn = "postgres:postgres@localhost:5432/covid_db"
engine = create_engine(f'postgresql://{conn}')

In [51]:
state_df.to_sql(name='us_state_pop',con=engine, if_exists='append', index=False)
covid_df.to_sql(name="daily_covid_stats", con=engine, if_exists='append',index=False)
combined_df.to_sql(name='final_covid_stats',con=engine,if_exists='append',index=False)


ProgrammingError: (psycopg2.errors.UndefinedColumn) column "2019_pop" of relation "us_state_pop" does not exist
LINE 1: INSERT INTO us_state_pop (state, name, "2019_pop") VALUES ('...
                                               ^

[SQL: INSERT INTO us_state_pop (state, name, "2019_pop") VALUES (%(state)s, %(name)s, %(2019_pop)s)]
[parameters: ({'state': 'AL', 'name': 'Alabama', '2019_pop': 4903185}, {'state': 'AK', 'name': 'Alaska', '2019_pop': 731545}, {'state': 'AZ', 'name': 'Arizona', '2019_pop': 7278717}, {'state': 'AR', 'name': 'Arkansas', '2019_pop': 3017804}, {'state': 'CA', 'name': 'California', '2019_pop': 39512223}, {'state': 'CO', 'name': 'Colorado', '2019_pop': 5758736}, {'state': 'CT', 'name': 'Connecticut', '2019_pop': 3565287}, {'state': 'DE', 'name': 'Delaware', '2019_pop': 973764}  ... displaying 10 of 51 total bound parameter sets ...  {'state': 'WI', 'name': 'Wisconsin', '2019_pop': 5822434}, {'state': 'WY', 'name': 'Wyoming', '2019_pop': 578759})]
(Background on 

In [47]:
result = engine.execute('SELECT * FROM final_covid_stats;')

In [50]:
print(result.fetchone())

engine.dispose()

None
